# --- Feature Engineering ---

### Carga de datos limpios

In [76]:
import pandas as pd
import numpy as np

df_visits = pd.read_csv('../data/processed/visits_clean.csv', parse_dates=['start_ts', 'end_ts'])
df_orders = pd.read_csv('../data/processed/orders_clean.csv', parse_dates=['buy_ts'])
df_costs = pd.read_csv('../data/processed/costs_clean.csv', parse_dates=['dt'])

### Variables de comportamiento

In [77]:
# Número total de compras por usuario
compras_por_usuario = df_orders.groupby('uid').size().rename('n_compras')

# Total gastado por usuario
gasto_total = df_orders.groupby('uid')['revenue'].sum().rename('revenue_total')

# AOV (Average Order Value)
aov = (gasto_total / compras_por_usuario).rename('aov')

# Fecha de primera compra
primera_compra = df_orders.groupby('uid')['buy_ts'].min().rename('fecha_primera_compra')

# Fecha de última compra
ultima_compra = df_orders.groupby('uid')['buy_ts'].max().rename('fecha_ultima_compra')

# Diferencia en días entre primera y última compra
dias_actividad = (ultima_compra - primera_compra).dt.days.rename('dias_entre_compras')

# Construcción del dataframe de comportamiento inicial
df_behavior = pd.concat([
    compras_por_usuario, gasto_total, aov,
    primera_compra, ultima_compra, dias_actividad
], axis=1).reset_index()

# %%
# Indicador de retención: si hubo una segunda compra al menos 30 días después de la primera
df_behavior['is_retained'] = (
    (df_behavior['n_compras'] > 1) & (df_behavior['dias_entre_compras'] >= 30)
).astype(int)

# Gasto mensual promedio (revenue total / meses activos)
df_behavior['meses_activo'] = df_behavior['dias_entre_compras'] / 30
df_behavior['meses_activo'] = df_behavior['meses_activo'].replace(0, 1)  # evitar división por cero
df_behavior['gasto_mensual_promedio'] = df_behavior['revenue_total'] / df_behavior['meses_activo']

# Frecuencia de compra (días promedio entre compras)
df_behavior['frecuencia_compras'] = df_behavior['dias_entre_compras'] / df_behavior['n_compras']

In [78]:
# Impresión
df_behavior

,uid,n_compras,revenue_total,aov,fecha_primera_compra,fecha_ultima_compra,dias_entre_compras,is_retained,meses_activo,gasto_mensual_promedio,frecuencia_compras
0,313578113262317,1,0.55,0.550000,2018-01-03 21:51:00,2018-01-03 21:51:00,0,0,1.000000,0.55,0.000000
1,1575281904278712,2,3.05,1.525000,2017-06-03 10:13:00,2017-06-03 17:39:00,0,0,1.000000,3.05,0.000000
2,2429014661409475,1,73.33,73.330000,2017-10-11 18:33:00,2017-10-11 18:33:00,0,0,1.000000,73.33,0.000000
3,2464366381792757,1,2.44,2.440000,2018-01-28 15:54:00,2018-01-28 15:54:00,0,0,1.000000,2.44,0.000000
4,2551852515556206,2,10.99,5.495000,2017-11-24 10:14:00,2017-11-24 11:24:00,0,0,1.000000,10.99,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
36518,18445147675727495770,1,3.05,3.050000,2017-11-24 09:03:00,2017-11-24 09:03:00,0,0,1.000000,3.05,0.000000
36519,18445407535914413204,3,0.88,0.293333,2017-09-22 23:55:00,2017-09-25 15:56:00,2,0,0.066667,13.20,0.666667
36520,18445601152732270159,1,4.22,4.220000,2018-03-26 22:54:00,2018-03-26 22:54:00,0,0,1.000000,4.22,0.000000
36521,18446156210226471712,1,9.78,9.780000,2018-02-18 19:34:00,2018-02-18 19:34:00,0,0,1.000000,9.78,0.000000


### Variables temporales

In [79]:
# Obtener fecha de primera sesión por usuario
df_visits['fecha_primera_sesion'] = df_visits.groupby('uid')['start_ts'].transform('min')

# Extraer mes y día de la semana
df_visits['mes_primera_sesion'] = df_visits['fecha_primera_sesion'].dt.month
df_visits['dia_semana_primera_sesion'] = df_visits['fecha_primera_sesion'].dt.dayofweek

# Dataset temporal único por usuario
df_temporal = df_visits[['uid', 'fecha_primera_sesion', 'mes_primera_sesion', 'dia_semana_primera_sesion']].drop_duplicates('uid')

# ¿Fue fin de semana?
df_temporal['es_fin_de_semana_primera_sesion'] = df_temporal['dia_semana_primera_sesion'].isin([5, 6]).astype(int)

# Codificar estación del año (0: Invierno, 1: Primavera, 2: Verano, 3: Otoño)
def codificar_estacion(mes):
    if mes in [12, 1, 2]:
        return 0  # Invierno
    elif mes in [3, 4, 5]:
        return 1  # Primavera
    elif mes in [6, 7, 8]:
        return 2  # Verano
    else:
        return 3  # Otoño

df_temporal['estacion_primera_sesion'] = df_temporal['mes_primera_sesion'].apply(codificar_estacion)

In [80]:
# Impresión
df_temporal

,uid,fecha_primera_sesion,mes_primera_sesion,dia_semana_primera_sesion,es_fin_de_semana_primera_sesion,estacion_primera_sesion
0,16879256277535980062,2017-12-20 17:20:00,12,2,0,0
1,104060357244891740,2018-02-19 16:53:00,2,0,0,0
2,7459035603376831527,2017-07-01 01:54:00,7,5,1,2
3,16174680259334210214,2018-03-09 20:05:00,3,4,0,1
4,9969694820036681168,2017-12-27 14:06:00,12,2,0,0
...,...,...,...,...,...,...
358517,12734910664455613822,2018-04-23 21:11:00,4,0,0,1
358522,6761309174945977743,2017-11-08 13:39:00,11,2,0,3
358525,11102751930812818282,2017-07-31 12:13:00,7,0,0,2
358527,272012551460639309,2018-03-20 15:11:00,3,1,0,1


### Variables de marketing

In [81]:
# Primer dispositivo y fuente
primer_dispositivo = df_visits.sort_values('start_ts').groupby('uid')['device'].first().rename('primer_dispositivo')
primer_source = df_visits.sort_values('start_ts').groupby('uid')['source_id'].first().rename('primer_source')

df_marketing = pd.concat([primer_dispositivo, primer_source], axis=1).reset_index()

# Codificar dispositivo: 0=desktop, 1=touch
df_marketing['primer_dispositivo'] = df_marketing['primer_dispositivo'].map({'desktop': 0, 'touch': 1})

# Número total de sesiones (proxy de interacción con marketing)
sesiones_usuario = df_visits.groupby('uid').size().rename('n_sesiones')
df_marketing = df_marketing.merge(sesiones_usuario, on='uid', how='left')

In [82]:
# Impresión
df_marketing

,uid,primer_dispositivo,primer_source,n_sesiones
0,11863502262781,1,3,1
1,49537067089222,1,2,1
2,297729379853735,0,3,1
3,313578113262317,0,2,3
4,325320750514679,0,5,2
...,...,...,...,...
228164,18446403737806311543,0,5,1
228165,18446424184725333426,1,4,1
228166,18446556406699109058,1,3,1
228167,18446621818809592527,0,4,2


### LTV_180

In [83]:
# Juntar visitas y órdenes para obtener fecha de primera sesión
primer_sesion = df_visits.groupby('uid')['start_ts'].min().rename('fecha_primera_sesion')
df_orders = df_orders.merge(primer_sesion, on='uid')

# Calcular ingresos dentro de los 180 días posteriores a la primera sesión
df_orders['dentro_180'] = df_orders['buy_ts'] <= df_orders['fecha_primera_sesion'] + pd.Timedelta(days=180)
ltv_180 = df_orders[df_orders['dentro_180']].groupby('uid')['revenue'].sum().rename('LTV_180').reset_index()

### CAC_source_30

In [84]:
# Merge para saber en qué fuente ocurrió la primera conversión
df_orders_sources = df_orders.merge(df_visits[['uid', 'source_id']], on='uid', how='left')

# Calcular la fecha de primera conversión por source
primeras_conversiones = df_orders_sources.groupby('source_id')['buy_ts'].min().rename('fecha_primera_conversion').reset_index()

# Unir con los costos
df_costs = df_costs.merge(primeras_conversiones, on='source_id', how='left')

# Costos dentro de los 30 días posteriores a la primera conversión
df_costs['dentro_30'] = df_costs['dt'] <= df_costs['fecha_primera_conversion'] + pd.Timedelta(days=30)

# Costos por fuente en esos 30 días
costos_por_source = df_costs[df_costs['dentro_30']].groupby('source_id')['costs'].sum().rename('costos_totales').reset_index()

# Usuarios únicos por fuente
usuarios_por_source = df_orders_sources.groupby('source_id')['uid'].nunique().rename('n_usuarios').reset_index()

# CAC = costos / usuarios
df_cac = costos_por_source.merge(usuarios_por_source, on='source_id', how='inner')
df_cac['CAC_source_30'] = df_cac['costos_totales'] / df_cac['n_usuarios']
df_cac = df_cac[['source_id', 'CAC_source_30']]

### Variables adiconales para mejorar la predicción de LTV y CAC

In [85]:
# 1. Días entre primera visita y primera compra
primeras_visitas = df_visits.groupby('uid')['start_ts'].min().rename('start_ts')
primeras_compras = df_orders.groupby('uid')['buy_ts'].min().rename('buy_ts')

df_gap_conversion = pd.concat([primeras_visitas, primeras_compras], axis=1).dropna()
df_gap_conversion['dias_primera_sesion_a_primera_compra'] = (
    df_gap_conversion['buy_ts'] - df_gap_conversion['start_ts']
).dt.days

# Unimos a df_behavior
df_behavior = df_behavior.merge(df_gap_conversion[['dias_primera_sesion_a_primera_compra']], on='uid', how='left')

# 2. Duración promedio por sesión
df_visits['duracion_sesion'] = (df_visits['end_ts'] - df_visits['start_ts']).dt.total_seconds()
duracion_promedio = df_visits.groupby('uid')['duracion_sesion'].mean().rename('duracion_promedio_sesion')

# 3. Número de dispositivos distintos usados por usuario (proxy de omnicanalidad)
n_dispositivos = df_visits.groupby('uid')['device'].nunique().rename('n_dispositivos_distintos')

# 4. Días entre primera y última sesión (actividad general, no solo de compra)
dias_activo_sesion = df_visits.groupby('uid').agg(
    fecha_primera_sesion=('start_ts', 'min'),
    fecha_ultima_sesion=('end_ts', 'max')
)
dias_activo_sesion['dias_activo'] = (
    dias_activo_sesion['fecha_ultima_sesion'] - dias_activo_sesion['fecha_primera_sesion']
).dt.days
dias_activo_sesion = dias_activo_sesion[['dias_activo']]

# 5. Cohorte mensual de adquisición
cohort_mes = df_temporal[['uid', 'fecha_primera_sesion']].copy()

# Extraer año y mes como variables separadas numéricas
cohort_mes['cohort_year'] = cohort_mes['fecha_primera_sesion'].dt.year
cohort_mes['cohort_month'] = cohort_mes['fecha_primera_sesion'].dt.month

# (opcional) Elimina la fecha completa si ya no es útil
cohort_mes.drop(columns='fecha_primera_sesion', inplace=True)

In [86]:
df_orders

,buy_ts,revenue,uid,order_month,fecha_primera_sesion,dentro_180
0,2017-06-01 00:10:00,17.00,10329302124590727494,2017-06,2017-06-01 00:09:00,True
1,2017-06-01 00:25:00,0.55,11627257723692907447,2017-06,2017-06-01 00:14:00,True
2,2017-06-01 00:27:00,0.37,17903680561304213844,2017-06,2017-06-01 00:25:00,True
3,2017-06-01 00:29:00,0.55,16109239769442553005,2017-06,2017-06-01 00:14:00,True
4,2017-06-01 07:58:00,0.37,14200605875248379450,2017-06,2017-06-01 07:31:00,True
...,...,...,...,...,...,...
50410,2018-05-31 23:50:00,4.64,12296626599487328624,2018-05,2018-05-31 09:11:00,True
50411,2018-05-31 23:50:00,5.80,11369640365507475976,2018-05,2018-05-31 23:05:00,True
50412,2018-05-31 23:54:00,0.30,1786462140797698849,2018-05,2018-05-31 23:52:00,True
50413,2018-05-31 23:56:00,3.67,3993697860786194247,2018-05,2017-10-23 12:32:00,False


In [88]:
primer_sesion = (
    df_visits
    .groupby('uid')['start_ts']
    .min()
    .rename('fecha_primera_sesion')
)
# 2) Mézclala con df_orders y crea days_from_first
df_orders = df_orders.merge(primer_sesion, on='uid', how='left')
df_orders['days_from_first'] = (
    df_orders['buy_ts'] - df_orders['fecha_primera_sesion']
).dt.days

# 3) Define la función de pendiente
def compute_slope(days, revenue):
    # Requerimos al menos 2 órdenes y 2 días distintos
    if len(days) < 2 or days.nunique() < 2:
        return np.nan
    try:
        return np.polyfit(days, revenue, 1)[0]
    except np.linalg.LinAlgError:
        return np.nan

# 4) Agrupa por usuario y calcula trend_slope
trend = (
    df_orders
    .groupby('uid')
    .apply(lambda g: compute_slope(g['days_from_first'], g['revenue']))
    .reset_index(name='trend_slope')
)

# 5) Crea la bandera has_slope
trend['has_slope'] = trend['trend_slope'].notna().astype(int)



/var/folders/w6/tt5g659n54zdxvlqlfd5dy3c0000gn/T/ipykernel_98760/2559646874.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: compute_slope(g['days_from_first'], g['revenue']))


### Dataset final para el modelado

In [89]:
# Unimos todas las variables extra al dataset final
df_extras = pd.concat([
    duracion_promedio, n_dispositivos, dias_activo_sesion
], axis=1).reset_index()

df_final = df_behavior.merge(df_temporal, on='uid', how='left') \
                      .merge(df_marketing, on='uid', how='left') \
                      .merge(df_cac, left_on='primer_source', right_on='source_id', how='left') \
                      .drop(columns=['source_id']) \
                      .merge(cohort_mes[['uid', 'cohort_year', 'cohort_month']], on='uid', how='left') \
                      .merge(df_extras, on='uid', how='left') \
                      .merge(ltv_180, on='uid', how='left') \
                        .merge(trend[['uid', 'trend_slope', 'has_slope']], on='uid', how='left')

# Exportar dataset final
df_final.to_csv('../data/processed/modeling_dataset.csv', index=False)

In [90]:
# Impresión
df_final

,uid,n_compras,revenue_total,aov,fecha_primera_compra,fecha_ultima_compra,dias_entre_compras,is_retained,meses_activo,gasto_mensual_promedio,...,n_sesiones,CAC_source_30,cohort_year,cohort_month,duracion_promedio_sesion,n_dispositivos_distintos,dias_activo,LTV_180,trend_slope,has_slope
0,313578113262317,1,0.55,0.550000,2018-01-03 21:51:00,2018-01-03 21:51:00,0,0,1.000000,0.55,...,3,0.340746,2017,9,660.000000,1,173,0.55,NaN,0
1,1575281904278712,2,3.05,1.525000,2017-06-03 10:13:00,2017-06-03 17:39:00,0,0,1.000000,3.05,...,2,0.184264,2017,6,480.000000,1,0,3.05,NaN,0
2,2429014661409475,1,73.33,73.330000,2017-10-11 18:33:00,2017-10-11 18:33:00,0,0,1.000000,73.33,...,1,0.571038,2017,10,780.000000,1,0,73.33,NaN,0
3,2464366381792757,1,2.44,2.440000,2018-01-28 15:54:00,2018-01-28 15:54:00,0,0,1.000000,2.44,...,2,0.264780,2018,1,450.000000,1,0,2.44,NaN,0
4,2551852515556206,2,10.99,5.495000,2017-11-24 10:14:00,2017-11-24 11:24:00,0,0,1.000000,10.99,...,2,0.264780,2017,11,279.500000,1,0,10.99,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36518,18445147675727495770,1,3.05,3.050000,2017-11-24 09:03:00,2017-11-24 09:03:00,0,0,1.000000,3.05,...,1,0.264780,2017,8,0.000000,1,0,3.05,NaN,0
36519,18445407535914413204,3,0.88,0.293333,2017-09-22 23:55:00,2017-09-25 15:56:00,2,0,0.066667,13.20,...,3,0.571038,2017,9,1020.000000,2,3,0.88,-0.17,1
36520,18445601152732270159,1,4.22,4.220000,2018-03-26 22:54:00,2018-03-26 22:54:00,0,0,1.000000,4.22,...,3,0.340746,2017,8,3000.000000,1,231,NaN,NaN,0
36521,18446156210226471712,1,9.78,9.780000,2018-02-18 19:34:00,2018-02-18 19:34:00,0,0,1.000000,9.78,...,7,0.571038,2017,11,1577.142857,2,108,9.78,NaN,0


In [91]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36523 entries, 0 to 36522
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   uid                                   36523 non-null  uint64        
 1   n_compras                             36523 non-null  int64         
 2   revenue_total                         36523 non-null  float64       
 3   aov                                   36523 non-null  float64       
 4   fecha_primera_compra                  36523 non-null  datetime64[ns]
 5   fecha_ultima_compra                   36523 non-null  datetime64[ns]
 6   dias_entre_compras                    36523 non-null  int64         
 7   is_retained                           36523 non-null  int64         
 8   meses_activo                          36523 non-null  float64       
 9   gasto_mensual_promedio                36523 non-null  float64       
 10